# Battery Capacity Prediction with DNN
This notebook trains a deep neural network using TensorFlow to predict the State of Charge (SOC) of a battery under different temperature conditions.

In [ ]:
!pip install tensorflow numpy pandas scipy matplotlib

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, InputLayer, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
# Load training MAT file
mat_data = scipy.io.loadmat('/content/drive/My Drive/CHEMINFOFINAL/train.mat')
print('Keys in MAT file:', mat_data.keys())
X_data = mat_data['X'].T
Y_data = mat_data['Y'].T.flatten()

In [ ]:
# Convert to DataFrame and normalize features
df = pd.DataFrame(np.hstack((X_data, Y_data.reshape(-1, 1))))
features = df.iloc[:, :-1].values
targets = df.iloc[:, -1].values
mean = features.mean(axis=0)
std = features.std(axis=0)
features = (features - mean) / std
val_split = int(0.8 * len(features))
train_features, train_targets = features[:val_split], targets[:val_split]
val_features, val_targets = features[val_split:], targets[val_split:]

In [ ]:
# Define DNN model
model = Sequential([
    InputLayer(input_shape=(train_features.shape[1],)),
    Dense(55),
    Activation('tanh'),
    Dense(55),
    LeakyReLU(alpha=0.3),
    Dense(1),
    Activation('relu')
])

def scheduler(epoch, lr):
    if epoch in [400, 800]:
        return lr * 0.1
    return lr
lr_scheduler = LearningRateScheduler(scheduler)

model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])

In [ ]:
history = model.fit(
    train_features, train_targets,
    epochs=1200,
    batch_size=1,
    validation_data=(val_features, val_targets),
    callbacks=[lr_scheduler],
    verbose=1
)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss', linestyle='dashed')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training Progress')
plt.legend()
plt.show()

In [ ]:
# Test data
test_files = {
    'n10degC': '/content/drive/My Drive/CHEMINFOFINAL/01_TEST_LGHG2@n10degC_Norm_(05_Inputs).mat',
    '0degC': '/content/drive/My Drive/CHEMINFOFINAL/02_TEST_LGHG2@0degC_Norm_(05_Inputs).mat',
    '10degC': '/content/drive/My Drive/CHEMINFOFINAL/03_TEST_LGHG2@10degC_Norm_(05_Inputs).mat',
    '25degC': '/content/drive/My Drive/CHEMINFOFINAL/04_TEST_LGHG2@25degC_Norm_(05_Inputs).mat'
}
predictions = {}
targets = {}
for temp, file in test_files.items():
    mat_data = scipy.io.loadmat(file)
    X_test = mat_data['X'].T
    Y_test = mat_data['Y'].T.flatten()
    Y_pred = model.predict(X_test, batch_size=1)
    predictions[temp] = Y_pred.flatten()
    targets[temp] = Y_test

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
temps = ['n10degC', '0degC', '10degC', '25degC']
for i, ax in enumerate(axes.flat):
    temp = temps[i]
    ax.plot(predictions[temp], label='Predicted', color='blue')
    ax.plot(targets[temp], label='Target', color='orange')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('SOC')
    ax.set_title(temp)
    ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
rmse_values = []
max_errors = []
for temp in temps:
    rmse = np.sqrt(mean_squared_error(targets[temp], predictions[temp])) * 100
    max_error = np.max(np.abs(targets[temp] - predictions[temp])) * 100
    rmse_values.append(rmse)
    max_errors.append(max_error)
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
axes[0].bar(temps, rmse_values, color='blue')
axes[0].set_ylabel('RMSE (%)')
axes[0].set_xlabel('Temperature (C)')
axes[1].bar(temps, max_errors, color='red')
axes[1].set_ylabel('MAX (%)')
axes[1].set_xlabel('Temperature (C)')
plt.tight_layout()
plt.show()